# Building the base tables

This is the notebook that was used to prepare the base tables for the pipeline in the paper "A Benchmarking Data Lake 
for Join Discovery and Learning with Relational Data".

All datasets are inspired from those used in the paper "[Relational data embeddings for feature enrichment with background information](https://hal.science/hal-03848124/file/main.pdf)", although some of the matching operations are slightly different.

To have more manageable and meaningful tables, some pre-processing is applied to each dataset to reduce overly noisy and 
irrelevant attributes. 

All tables are saved as parquet files for later use in the benchmark pipeline. 

Datasets:
- [7+ Million Company Dataset](https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset)
- [US Accidents (2016 - 2021)](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents)
- [US Presidential elections](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX)

In [1]:
cd ..

/home/soda/rcappuzz/work/prepare-data-lakes


In [2]:
import polars as pl
import pandas as pd
from pathlib import Path
import src.yago.utils as utils
import numpy as np
import datetime

Loading YAGO fact triplets to drop entities not found in the KB.

Note that this step requires the YAGO3 3.0.3 data (accessible 
[here](https://yago-knowledge.org/downloads/yago-3), under "2022 Revival Data") to have been downloaded. Replace the 
value in `yago_path` with the path to the YAGO facts on your machine. 

In [3]:
data_dir = Path("data/base_tables")

yago_path = Path("/storage/store3/work/jstojano/yago3/")
facts_path = Path(yago_path, "facts_parquet/yago_updated_2022_part2")
fname = "yagoFacts"
yagofacts_path = Path(facts_path, f"{fname}.tsv.parquet")
yagofacts_categorical = utils.import_from_yago(yagofacts_path, engine="polars")
fname = "yagoLiteralFacts"
yagoliteralfacts_path = Path(facts_path, f"{fname}.tsv.parquet")
yagofacts_numerical = utils.import_from_yago(yagoliteralfacts_path, engine="polars")
fname = "yagoDateFacts"
yagodatefacts_path = Path(facts_path, f"{fname}.tsv.parquet")
yagofacts_dates = utils.import_from_yago(yagodatefacts_path, engine="polars")

yagofacts = pl.concat(
    [
        yagofacts_categorical,
        yagofacts_numerical,
        yagofacts_dates
    ]
)

# US Accidents dataset

Archive `us-accidents.zip` contains the file `US_Accidents_Dec21_updated.csv`, 
which was renamed manually to `us-accidents.csv` for simplicity. 

Some of the steps require the State codes reported in `data/state_codes.csv`. 

In [31]:
dataset_dir = Path(data_dir, "us-accidents")
df = pl.read_csv(Path(dataset_dir, "us-accidents.csv"))
df = df.rename({"State": "Code"})
state_codes_path =  Path(dataset_dir,"state_codes.csv")
state_codes = pl.read_csv(state_codes_path)
df = df.join(
    state_codes, on="Code"
)

Adding a new column, `col_to_embed`, that formats the city and state name to have
the same format that is found in YAGO. 

In [32]:
df = df.with_columns(
    ("<" + pl.col("City") + ",_"+ pl.col("State") + ">").alias("col_to_embed")
)

Filtering out the rows not found in `yagofacts["subject"]`.

In [33]:
df_filtered=df.lazy().filter(
    pl.col("col_to_embed").is_in(
        yagofacts["subject"]
    )
).collect()

For the preparation, we select only the accidents whose `Start_Time` is between 2019-01-01 and 2019-12-31. 

In [34]:
start_date = datetime.date(2019, 1,1)
end_date = datetime.date(2019, 12, 31)
df_by_year = df_filtered.filter(
    (pl.col("Start_Time").str.to_datetime()>start_date) & (pl.col("Start_Time").str.to_datetime()<end_date)
)

Counting the number of accidents (log10) per county in the given year.

In [35]:
df_counts_by_year=df_by_year.groupby(
    [
        "col_to_embed", "City", "Code"
    ]
    ).count().select(
        pl.col("col_to_embed"),
        pl.col("count").alias("target").log10()
    )
df_counts_by_year

col_to_embed,target
str,f64
"""<Lansing,_Mich…",2.164353
"""<Greenwich,_Co…",1.623249
"""<Lakeville,_Mi…",1.934498
"""<Bellaire,_Tex…",1.477121
"""<Ogden,_Utah>""",2.970347
"""<Warrenton,_Or…",2.113943
"""<Cotati,_Calif…",1.531479
"""<Richmond,_Cal…",2.457882
"""<Delhi,_Califo…",1.591065


### Preprocessing

In the KEN paper, everything except the county was dropped. Here, we need to rely on the information still found in the 
table, so we need to to reduce the table to have only one sample for each row. It is necessary to aggregate any information
that has granularity smaller than "county". 

The overall number of columns is also reduced from the original. 

To aggregate all the values, the `mode` is used to select the most frequent categorical value, while the `mean` is used
on the numerical attributes. 

The resulting table is saved in `us-accidents-yadl.parquet`.

In [36]:
df_final = df_counts_by_year.join(
    df_by_year, 
    on="col_to_embed",
    how="inner"
).groupby("col_to_embed").agg(
    pl.col("target").mean(),
    pl.col("County").mode().first(),
    pl.col("Code").mode().first(),
    pl.col("Severity").mean(),
    pl.col("Zipcode").mode().first(),
    pl.col("Country").mode().first(),
    pl.col("Airport_Code").mode().first(),
    pl.col("Visibility(mi)").mean(),
    pl.col("Sunrise_Sunset").mode().first(),
    pl.col("Civil_Twilight").mode().first(),
    pl.col("State").mode().first(),    
)
df_final.write_parquet(Path(dataset_dir, "us-accidents-yadl.parquet"))


Here we check the percentage of missing values in each column. 

In [37]:
df_final.null_count()/len(df_final)*100

col_to_embed,target,County,Code,Severity,Zipcode,Country,Airport_Code,Visibility(mi),Sunrise_Sunset,Civil_Twilight,State
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248947,1.876676,0.0,0.0,0.0


# Company Employees Dataset

The "Company Employees" dataset contains information about companies. The prediction target in this case is the number 
of employees of a company. We filter the dataset to select only companies with at least 1000 employees.

In [17]:
dataset_dir = Path(data_dir, "company-employees")
df = pl.read_csv(Path(dataset_dir, "companies_sorted.csv"))
df_selected = df.filter(
    pl.col("current employee estimate") >= 1000
)

Adding a new column to `yagofacts` with lowercased subjects. 

In [18]:
yagofacts = yagofacts.with_columns(
    pl.col("subject").str.to_lowercase().alias("subject_formatted")
)
df_filtered=df_selected.lazy().with_columns(
    ("<" + pl.col("name").str.to_lowercase().str.replace(" ", "_") + ">").alias("formatted_name")
).filter(
    pl.col("formatted_name").is_in(yagofacts["subject_formatted"])
).collect()

Not all the company names can be matched with this simple heuristic due to the fact that the name in the base table has 
a format different from that in the knowledge base. 

In [19]:
df_selected.lazy().with_columns(
    ("<" + pl.col("name").str.to_lowercase().str.replace(" ", "_") + ">").alias("formatted_name")
).filter(
    ~pl.col("formatted_name").is_in(yagofacts["subject_formatted"])
).limit(20).select(
                   pl.col("name"),
                   pl.col("formatted_name")).collect()

name,formatted_name
str,str
"""tata consultan…","""<tata_consulta…"
"""us army""","""<us_army>"""
"""ey""","""<ey>"""
"""cognizant tech…","""<cognizant_tec…"
"""united states …","""<united_states…"
"""pwc""","""<pwc>"""
"""citi""","""<citi>"""
"""bank of americ…","""<bank_of ameri…"
"""jpmorgan chase…","""<jpmorgan_chas…"


In [20]:
print(yagofacts.lazy().filter(
    pl.col("subject").str.contains("Apple")
).limit(1).collect())

print(yagofacts.lazy().filter(
    pl.col("subject").str.to_lowercase().str.contains("united_states_army")
).collect())

shape: (1, 6)
┌───────────────────┬──────────────┬───────────┬───────────────────┬────────────┬──────────────────┐
│ id                ┆ subject      ┆ predicate ┆ cat_object        ┆ num_object ┆ subject_formatte │
│ ---               ┆ ---          ┆ ---       ┆ ---               ┆ ---        ┆ d                │
│ str               ┆ str          ┆ str       ┆ str               ┆ f64        ┆ ---              │
│                   ┆              ┆           ┆                   ┆            ┆ str              │
╞═══════════════════╪══════════════╪═══════════╪═══════════════════╪════════════╪══════════════════╡
│ <id_22jfQW8Rcs_5y ┆ <Apple_Inc.> ┆ <owns>    ┆ <Apple_Icon_Image ┆ null       ┆ <apple_inc.>     │
│ H_L2hDNlxUyQ>     ┆              ┆           ┆ _format>          ┆            ┆                  │
└───────────────────┴──────────────┴───────────┴───────────────────┴────────────┴──────────────────┘
shape: (1_086, 6)
┌─────────────────┬────────────────┬────────────────┬──────

Here we prepare a mapping between the name in the original dataset and the match found in YAGO.
Note that there is a relatively low recall.

In [21]:
mapping_name_subject = df_filtered.lazy().join(
    yagofacts.lazy(),
    left_on="formatted_name",
    right_on="subject_formatted"
).select(
    [
        pl.col("name"),
        pl.col("formatted_name"),
        pl.col("subject")
    ]
).unique().collect()


Joining on with the mapping on `formatted_name` to guarantee that col `col_to_embed` uses the same format (and 
capitalization) used in YAGO. 

In [22]:
df_final = df_filtered.join(
    mapping_name_subject, on="formatted_name"
).select(
    [
        pl.col("name").alias("raw_entities"),
        pl.col("subject").alias("col_to_embed"),
        pl.col("current employee estimate").alias("target").log10()
    ]
)
df_final


raw_entities,col_to_embed,target
str,str,f64
"""ibm""","""<IBM>""",5.437825
"""accenture""","""<Accenture>""",5.280326
"""hewlett-packar…","""<Hewlett-Packa…",5.107047
"""walmart""","""<Walmart>""",5.081898
"""microsoft""","""<Microsoft>""",5.065191
"""at&t""","""<AT&T>""",5.061407
"""wells fargo""","""<Wells_Fargo>""",5.039541
"""infosys""","""<Infosys>""",5.020162
"""deloitte""","""<Deloitte>""",5.017501


In [23]:
df_final.write_parquet(Path(dataset_dir, "company-employees-target.parquet"))

Joining the measure target on the base table. Some unnecessary columns are dropped from the table, then it is saved on disk.


In [26]:
df_prepared = df_filtered.lazy().join(
    df_final.lazy(),
    left_on="name",
    right_on="raw_entities",
    how="inner"
).drop(
    "",
    "formatted_name",
    "current employee estimate",
    "total employee estimate",
).collect()
df_prepared.write_parquet(Path(dataset_dir, "company-employees-yadl.parquet"))

Checking the percentage of missing values by column.

In [29]:
df_prepared.null_count()/len(df_prepared)*100

name,domain,year founded,industry,size range,locality,country,linkedin url,col_to_embed,target
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,4.503056,17.72274,0.0,0.0,10.871663,9.488582,0.0,0.0,0.0


# US Presidential elections

The US Presidential Elections dataset contains the vote fraction for each candidate in each county in the US for the 
Presidential Elections in the period 2000-2020. We focus on the 2020 election, thus we select only the rows relative to 
that. 

As usual, the column `col_to_embed` is formatted to follow YAGO's string format. 

In [38]:
dataset_dir = Path(data_dir, "presidential-results")
df = pl.read_csv(Path(dataset_dir, "presidential-results.csv"), infer_schema_length=0)
df = df.to_pandas()

In [39]:
df = df[df["year"] == "2020"]
df["county_name"] = df["county_name"].str.title()
df["state"] = df["state"].str.title()
df["col_to_embed"] = "<" + df["county_name"] + "_County,_" + df["state"] + ">"
df["col_to_embed"] = df["col_to_embed"].str.replace(" ", "_")
df["target"] = np.log10(df["candidatevotes"].astype(int) + 1)
df["raw_entities"] = df["county_name"] + " " + df["state"]
df.loc[df["state"]=="Louisiana", "col_to_embed"]=df.loc[df["state"]=="Louisiana", "col_to_embed"].str.replace("County", "Parish")
# df = df[["raw_entities", "col_to_embed", "party", "target"]]
df.dropna(inplace=True)


Some columns are dropped because they are either not useful, or would leak information about the target. 

In [40]:
df_final = df.drop(
    ["raw_entities", "candidatevotes", "county_fips", "office", "year", "totalvotes", "version", "mode"], axis=1
    )
df_final.to_parquet(Path(dataset_dir, "us-presidential-results-yadl.parquet"), index=False)

Checking the number of missing values by column. 

In [42]:
df_final.isna().sum()

state           0
state_po        0
county_name     0
candidate       0
party           0
col_to_embed    0
target          0
dtype: int64